In [1]:
import papermill as pm
from nbconvert import PDFExporter
import nbformat

from orbit_generation.experiment import generate_parameter_sets

Parameters

In [ ]:
params = {
    # Data
    'data_used': 'EM_N_fix_1500',
    'families_to_discard': [0, 2, 4, 10, 20],
    'seq_len': 100,
    'feature_dim': 7,
    
    # Training
    'epochs': 50,
    'val_split': 0.05,
    'batch_size': 32,
    'lr': 0.001,
    
    # Model
    'model_name': ['vae_conv5', 'inception_time_wp_vae'],
    'latent_dim': [2, 7],
    'beta': [0.001, 0.2, 0.5, 1, 1.5, 2, 10],
    
    # Convergence
    'max_iter_convergence': 20,
    'input_seq_len_convergence': 1, ### make experiment first
    
    
    # Evaluation
    'samples_to_generate': 100,
    'distance_metric': 'euclidean'
}

In [3]:
model_specific_params = {
    'vae_conv5': {
        'dropout_rate': 0.2
    },
    'inception_time_wp_vae': {
        'n_filters': 32,
        'kernel_sizes': [3, 7, 13],
        'bottleneck_channels': 32
    }
}

Parameter Set

In [4]:
parameter_sets = generate_parameter_sets(params, model_specific_params)

In [5]:
len(parameter_sets)

140

In [ ]:
import os
import json
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import papermill as pm
import nbformat
from nbconvert.exporters import PDFExporter
from nbconvert.preprocessors import ExecutePreprocessor

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [8]:
def execute_notebook(params, i, notebook_to_execute, output_dir):
    try:
        # Generate output filenames
        base_name = os.path.splitext(os.path.basename(notebook_to_execute))[0]
        pdf_file = os.path.join(output_dir, f"{base_name}_execution_{i}.pdf")
        
        # Check if PDF already exists
        if os.path.exists(pdf_file):
            logging.info(f"Skipping execution {i}, PDF already exists.")
            return i
        
        # Read the notebook
        with open(notebook_to_execute, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)

        # Find the parameters cell and update it
        params_cell_found = False
        for index, cell in enumerate(nb.cells):
            if cell.cell_type == 'code' and cell.source.startswith('# parameters'):
                # Update the cell source with new parameter values
                new_source = "# parameters\n"
                for key, value in params.items():
                    new_source += f"{key} = {repr(value)}\n"
                cell.source = new_source
                params_cell_found = True
                
                # Add a cell to print out and verify the parameters
                verify_params_cell = nbformat.v4.new_code_cell(
                    source="print('Injected parameters:', {" + 
                           ", ".join(f"'{k}': {k}" for k in params.keys()) + 
                           "})"
                )
                nb.cells.insert(index + 1, verify_params_cell)
                break

        if not params_cell_found:
            raise ValueError("Parameters cell not found in the notebook")

        # Execute only the parameters cell and the verification cell
        ep = ExecutePreprocessor(timeout=600, kernel_name='pytorch')
        ep.preprocess(nb, {'metadata': {'path': os.path.dirname(notebook_to_execute)}})

        # Check if parameters were properly injected
        if len(nb.cells) > index + 1 and nb.cells[index + 1].outputs:
            injected_params = nb.cells[index + 1].outputs[0].text
            logging.info(f"Injected parameters for execution {i}: {injected_params}")
            
            # Verify that all parameters are present
            for key in params.keys():
                if key not in injected_params:
                    raise ValueError(f"Parameter '{key}' was not properly injected")
        else:
            raise ValueError("Failed to verify injected parameters")

        # Now execute the rest of the notebook
        nb = pm.execute_notebook(
            nb,
            None,
            parameters={},  # Empty dict since we've already injected the parameters
            kernel_name='pytorch',
            start_timeout=600
        )
        
        # Create a PDF exporter
        pdf_exporter = PDFExporter()
        pdf_exporter.exclude_input = True
        
        # Convert to PDF
        pdf_data, _ = pdf_exporter.from_notebook_node(nb)
        
        # Save the PDF
        with open(pdf_file, 'wb') as f:
            f.write(pdf_data)
        
        logging.info(f"Completed execution {i}")
        return i

    except Exception as e:
        logging.error(f"Error in execution {i}: {str(e)}")
        logging.error(f"Parameters used: {params}")
        import traceback
        logging.error(f"Traceback: {traceback.format_exc()}")
        return None

In [ ]:
#| export
def process_parameter_sets(parameter_sets, notebook_to_execute, output_dir, checkpoint_file, max_workers=4):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize or load checkpoint
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            checkpoint = json.load(f)
    else:
        checkpoint = {'completed': []}
    
    # Ensure checkpoint is a dictionary with a 'completed' key
    if not isinstance(checkpoint, dict) or 'completed' not in checkpoint:
        checkpoint = {'completed': []}
    
    # Filter out already completed executions
    remaining_executions = [i for i in range(len(parameter_sets)) if i not in checkpoint['completed']]
    
    logging.info(f"Starting execution. {len(remaining_executions)} executions remaining.")
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(execute_notebook, parameter_sets[i], i, notebook_to_execute, output_dir) 
                   for i in remaining_executions]
        
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                logging.info(f"Execution {result} completed successfully.")
                # Update checkpoint
                checkpoint['completed'].append(result)
                with open(checkpoint_file, 'w') as f:
                    json.dump(checkpoint, f)
            else:
                logging.warning("An execution failed.")
    
    logging.info("All executions completed.")


# Usage
if __name__ == "__main__":
    notebook_to_execute = '03_01_generative_discovery.ipynb'
    output_dir = "../experiments/03_01_generative_discovery"
    checkpoint_file = '../experiments/experiment_checkpoint.json'
    
    process_parameter_sets(
        parameter_sets,
        notebook_to_execute=notebook_to_execute,
        output_dir=output_dir,
        checkpoint_file=checkpoint_file,
        max_workers=1
    )

2024-11-11 14:10:42,565 - INFO - Starting execution. 140 executions remaining.
2024-11-11 14:17:30,722 - ERROR - Error in execution 0: An error occurred while executing the following cell:
------------------
# Experiment
experiment_folder = setup_new_experiment(params, experiments_folder)
images_folder = os.path.join(experiment_folder, 'images')
if not os.path.exists(images_folder):
    os.makedirs(images_folder)
experiment_id = os.path.basename(experiment_folder).split('_')[1]
------------------


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[10], line 2
      1 # Experiment
----> 2 experiment_folder = setup_new_experiment(params, experiments_folder)
      3 images_folder = os.path.join(experiment_folder, 'images')
      4 if not os.path.exists(images_folder):

NameError: name 'params' is not defined

2024-11-11 14:17:30,726 - ERROR - Parameters used: {'data_used': 'EM_N_f

In [ ]:
import os

notebook_to_execute = '../experiments/experiment_checkpoint.json'

if os.path.isfile(notebook_to_execute):
    print("File exists.")
else:
    print("File does not exist.")


File does not exist.
